In [20]:
import wandb
import configparser
import torch
import torch.nn as nn
from torchvision import datasets, transforms
import wandb

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Config parser 불러오기

In [3]:
def load_config(config_path:str) -> dict:
    config = configparser.ConfigParser()    
    config.read(config_path, encoding='utf-8') 
    # print(config.sections())
    
    new_dict = {}
    for section in config.sections():
        new_dict[section] = dict()
        for value in config[section]:
            try:
                new_value = float(config[section][value])
                new_dict[section][value] = new_value
            except ValueError:
                new_value = config[section][value].lower()
                if new_value in ['y','yes','true','t','on']:
                    new_dict[section][value] = True
                elif new_value in ['n','no','false','f','off']:
                    new_dict[section][value] = False
                else:
                    new_dict[section][value] = [int(i) for i in eval(config[section][value])]
    return new_dict

In [4]:
config = configparser.ConfigParser()    
config.read("config_cifar.ini", encoding='utf-8') 

batch_size = config['TRAINING'].getint('batch_size')
epochs = config['TRAINING'].getint('epoch')
learning_rate = config['TRAINING'].getfloat('learning_rate')
optimizer_type = config['TRAINING']['optimizer'].lower()
loss_function_type = config['TRAINING']['loss_function'].lower()

## Wandb initialize

In [21]:
wandb.init(
    # set the wandb project where this run will be logged
    project="CIFAR-10-classification",
    name = 'practice_1',
    # track hyperparameters and run metadata
    config=config
    )

In [6]:
train_dataset = datasets.CIFAR10(root="./data/",
                                 train=True,
                                 download=True,
                                 transform=transforms.ToTensor())

test_dataset = datasets.CIFAR10(root="./data/",
                                train=False,
                                download=True,
                                transform=transforms.ToTensor())


Files already downloaded and verified
Files already downloaded and verified


In [7]:
BATCH_SIZE = batch_size
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=BATCH_SIZE,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=BATCH_SIZE,
                                          shuffle=False)

print(train_loader.dataset)

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data/
    Split: Train
    StandardTransform
Transform: ToTensor()


In [8]:
for (X_train, Y_train) in train_loader:
    print(f"X_train: {X_train.size()} type: {X_train.type()}")
    print(f"Y_train: {Y_train.size()} type: {Y_train.type()}")
    break

X_train: torch.Size([32, 3, 32, 32]) type: torch.FloatTensor
Y_train: torch.Size([32]) type: torch.LongTensor


In [9]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=8,
            kernel_size=3,
            padding=1)
        self.conv2 = nn.Conv2d(
            in_channels=8,
            out_channels=16,
            kernel_size=3,
            padding=1)
        self.pool = nn.MaxPool2d(
            kernel_size=2,
            stride=2
        )
        self.fc1 = nn.Linear(8 * 8 * 16, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = torch.relu(x)
        x = self.pool(x)

        x = x.view(-1, 8 * 8 * 16)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.fc3(x)
        x = torch.log_softmax(x, dim=1)
        return x

In [10]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

print(f"Using PyTorch version: {torch.__version__}, Device: {DEVICE}")

model = CNN().to(DEVICE)

if optimizer_type=='adam':
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
elif optimizer_type =='sgd':
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
else:
    print("Invalid optimizer type!!")
    raise ValueError

if loss_function_type == 'crossentrophy':
    criterion = nn.CrossEntropyLoss()
elif loss_function_type == 'bce':
    criterion = nn.BCEWithLogitsLoss()
else:
    print("Invalid loss fuction type!!")
    raise ValueError

Using PyTorch version: 2.1.2, Device: cuda


In [17]:
def train(model, train_loader, optimizer, log_interval):
    model.train()
    train_loss=0
    correct =0
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print(f"train Epoch: {Epoch} [{batch_idx * len(image)}/{len(train_loader.dataset)}({100. * batch_idx / len(train_loader):.0f}%)]\tTrain Loss: {loss.item()}")

        train_loss+=loss.item()
        prediction = output.max(1, keepdim=True)[1]
        correct += prediction.eq(label.view_as(prediction)).sum().item()
    train_loss /= len(train_loader.dataset)
    train_accuracy = 100. * correct / len(train_loader.dataset)
    return train_loss, train_accuracy

In [18]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim=True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [19]:
wandb.watch(model)

EPOCHS = epochs
for Epoch in range(1, EPOCHS + 1):
    train_loss, train_accuracy = train(model, train_loader, optimizer, log_interval=200)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print(f"\n[EPOCH: {Epoch}]\tTest Loss: {test_loss:.4f}\tTest Accuracy: {test_accuracy} % \n")

    wandb.log({"Train_loss":train_loss,"Train_acc":train_accuracy,"Test_loss":test_loss,"Test_acc":test_accuracy})
wandb.finish()

train Epoch: 1 [0/50000(0%)]	Train Loss: 1.5963411331176758
train Epoch: 1 [6400/50000(13%)]	Train Loss: 1.4888315200805664
train Epoch: 1 [12800/50000(26%)]	Train Loss: 1.3174375295639038
train Epoch: 1 [19200/50000(38%)]	Train Loss: 1.3333320617675781
train Epoch: 1 [25600/50000(51%)]	Train Loss: 1.6345903873443604
train Epoch: 1 [32000/50000(64%)]	Train Loss: 1.347216010093689
train Epoch: 1 [38400/50000(77%)]	Train Loss: 1.2516660690307617
train Epoch: 1 [44800/50000(90%)]	Train Loss: 1.4458508491516113

[EPOCH: 1]	Test Loss: 0.0420	Test Accuracy: 51.76 % 

train Epoch: 2 [0/50000(0%)]	Train Loss: 0.9427995681762695
train Epoch: 2 [6400/50000(13%)]	Train Loss: 1.2183690071105957
train Epoch: 2 [12800/50000(26%)]	Train Loss: 1.2432805299758911
train Epoch: 2 [19200/50000(38%)]	Train Loss: 1.3746626377105713
train Epoch: 2 [25600/50000(51%)]	Train Loss: 1.3797105550765991
train Epoch: 2 [32000/50000(64%)]	Train Loss: 1.1285067796707153
train Epoch: 2 [38400/50000(77%)]	Train Loss: 1.

Test_acc,▁▂▄▄▆▆▇▇█▇▇█▇▇█▇███▇
Test_loss,██▅▄▃▃▂▁▁▂▂▁▂▂▂▃▂▂▂▃
Train_acc,▁▂▃▄▄▅▅▆▆▆▇▇▇▇▇▇████
Train_loss,█▇▆▅▅▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁
Test_acc,62.94
Test_loss,0.0343
Train_acc,74.034
Train_loss,0.02309
